# Q1

## Instalando as dependências

In [ ]:
%pip install -q matplotlib
%pip install -q numpy

## Importando o numpy e o matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Criando as funções básicas

In [ ]:
def calcule_yPoints_linear_function(x: np.ndarray, bias: float, slope: float) -> np.ndarray:
    """Calcule the current y from f(x) = weight * slope + bias

    Args:
        x (np.ndarray | float): Current x in X axle
        bias (float): The 'b' in f(x) = ax + b
        slope (float): The 'a' in f(x) = ax + b

    Returns:
        np.ndarray: y value from the linear function
    """
    return bias + (slope * x)

def calcule_square_error(real_dots: np.ndarray[tuple], bias: float, slope: float) -> float:
    """Calcule the current square error

    Args:
        real_dots (np.ndarray[tuple]): Real points (x, y) collected out of IA
        bias (float): The 'b' in f(x) = ax + b
        slope (float): The 'a' in f(x) = ax + b

    Returns:
        float: calcule from square error
    """

    # x_real and y_real are real values collecteds
    # calcule the error of the linear regression in x_real
    x_real: np.ndarray = real_dots[:, 0]
    y_real: np.ndarray = real_dots[:, 1]

    # y_pred is the IA response
    y_pred: np.ndarray = calcule_yPoints_linear_function(x=x_real, bias=bias, slope=slope)

    error_array: np.ndarray = y_real - y_pred

    # all error array
    return np.sum(error_array ** 2)

def calcule_error_curve_values(x_ticks: np.ndarray, slope: float, real_dots: np.ndarray) -> np.ndarray:
    """Get all square error curve values in x linespace

    Args:
        x_ticks (np.ndarray): All x values to calculate (x axle)
        slope (float): The 'a' in f(x) = ax + b (weight)
        real_dots (np.ndarray): Real points (x, y) collected out of IA

    Returns:
        np.ndarray: All square error calculated
    """

    # x_ticks are all avaliable bias
    # bias = intecept
    y_errors_values: list[float] = [calcule_square_error(slope=slope, bias=bias, real_dots=real_dots) for bias in x_ticks]

    return np.array(y_errors_values)

def calcule_partial_derivative_bias(y_real: np.ndarray, y_pred: np.ndarray) -> float:
    """Get the partial derivative of square error curve

    Args:
        y_real (np.ndarray): Real points collected
        y_pred (np.ndarray): Linear Regression points (same lenght of points with `y_real`)

    Returns:
        float: Partial derivative
    """

    # -2 * (real_value - (bias + slope * x) + ... # x is a independent variable
    # this is the patial derivate
    # y_pred is a array with all values predicted by the linear regression
    # y_real is a arrrary with all real values
    return -2 * np.sum(y_real - y_pred)


def calcule_tan_line(square_error: float, y_real: np.ndarray, y_pred: np.ndarray, bias: float, tan_range: float = 1.5) -> tuple[np.ndarray, np.ndarray]:
    """Calcule the current tangent line of the square error curve

    Args:
        square_error (float): Error of the linear regression
        y_real (np.ndarray): Real y values
        y_pred (np.ndarray): Prediced y values by linear regression
        bias (float): X in square error curve or (bias of the linear regression)
        tan_range (float, optional): Range to calculate the tangent (x size). Defaults to 1.5.

    Returns:
        tuple[np.ndarray, np.ndarray]: Tuple with current x tangent values and y tangent values
    """

    # bias is a x variable in square error curve
    x_tan: np.ndarray = np.array([bias - tan_range, bias, bias + tan_range])

    # equation of the straight line
    # y = m * (x - x0) + y0
    y_tan: np.ndarray = calcule_partial_derivative_bias(y_real, y_pred) * (x_tan - bias) + square_error
    return (x_tan, y_tan)


## Funções para criar os gráficos

In [ ]:
from typing import Generator
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.lines import Line2D


def create_subplots() -> tuple[Figure, tuple[Axes, Axes]]:
    """Create subplots to show animations

    Returns:
        tuple[Figure, tuple[Axes, ...]]: The figure and a tuple with all lines
    """
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    return fig, axes

def config_axels(
        axes: tuple[Axes, Axes], 
        real_dots: np.ndarray,
        bias: float,
        slope: float,
        x_ticks_linear_regression: np.ndarray,
        ypoints_linear_regression: np.ndarray,
        x_ticks_error_curve: np.ndarray,
        ypoints_square_error_curve: np.ndarray,
        ) -> tuple:

    # linear regrassion
    ax1: Axes = axes[0]

    # square error
    ax2: Axes = axes[1]

    linear_regression, = ax1.plot(x_ticks_linear_regression, ypoints_linear_regression)

    # add individual real points
    ax1.scatter(x=real_dots[:, 0], y=real_dots[:, 1], label='Pontos Reais', color='green', s=15, marker="o")

    ax1.set_title("Linear Regression")
    ax1.set_xlabel("weight")
    ax1.set_ylabel("height")

    # define origin to (0, 0)
    ax1.set_xlim(left=0, right=x_ticks_linear_regression[-1])
    ax1.set_ylim(bottom=0)

    ax1.grid(visible=True, axis='both', alpha=0.5, color='gray')
    bias_text = ax1.text(
        0.1, # 10% x
        0.9, # 90% y
        f'Bias (intercept): {bias:.4f}',
        transform=ax1.transAxes, 
        fontsize=12, 
        bbox=dict(facecolor='white', alpha=0.7)
    )

    ax2.plot(x_ticks_error_curve, ypoints_square_error_curve, label='Square Error')
    ax2.set_title("Square Error")
    ax2.set_xlabel("Intercept (bias)")
    ax2.set_ylabel("Sum of Square Errors")

    square_error: float = calcule_square_error(real_dots, bias, 0.64)
    x_real =  real_dots[:, 0]
    y_real = real_dots[:, 1]
    square_error_curve_tangent, = ax2.plot(
        *calcule_tan_line(
            square_error,
            y_real,
            calcule_yPoints_linear_function(x_real, bias, slope),
            bias
        ),
        label='Derivative'
        )
    
    ax2.grid(visible=True, axis='both', alpha=0.5, color='gray')
    error_text = ax2.text(
        0.1,
        0.9,
        f'Square Error: {square_error:.4f}',
        transform=ax2.transAxes,
        fontsize=12,
        bbox=dict(facecolor='white', alpha=0.7)
    )
    
    current_error_point, = ax2.plot(
        [bias],
        [square_error],
        'ro',
        markersize=8,
        label='Current Error'
    )

    plt.legend()

    # define origin to (0, 0)
    ax2.set_xlim(left=0, right=x_ticks_error_curve[-1])
    ax2.set_ylim(bottom=0)

    return linear_regression, square_error_curve_tangent, current_error_point, bias_text, error_text

def gradient_descent_generator(
        initial_bias: float,
        slope: float,
        real_dots: np.ndarray,
        learn_rate: float,
        max_iterations: int,
    ) -> Generator[float]:
    """Generate a gradient descent to update the linear regression bias

    Args:
        initial_bias (float): First bias of the linear regression
        slope (float): The 'a' of linear equation: f(x) = ax + b
        real_dots (np.ndarray): Real dots of data
        learn_rate (float): Multiply factor of the de step size
        max_iterations (int): Limit of iterations

    Yields:
        Generator[float]: Generator with updated bias
    """

    current_bias: float = initial_bias
    for _ in range(max_iterations):
        x_real: np.ndarray = real_dots[:, 0]
        y_real: np.ndarray = real_dots[:, 1]
        y_pred: np.ndarray = calcule_yPoints_linear_function(
            x=x_real,
            bias=current_bias,
            slope=slope,
        )

        partial_derivative: float = calcule_partial_derivative_bias(
            y_real=y_real,
            y_pred=y_pred,
        )

        step_size: float = partial_derivative * learn_rate

        current_bias -= step_size
       
        if step_size == 0:
            break

        yield current_bias
        

def animate( 
        current_bias: float,
        x_ticks_linear_regression: np.ndarray,
        real_dots: np.ndarray,
        lines: tuple[Line2D, ...],
        slope: float = 0.0,
        tan_range: float = 1.5,
        ) -> tuple[Line2D, ...]:
    """Animate the new values of the linear regression and the square error function

    Args:
        current_bias (float): Current 'b' (intercept) of the linear regression: f(x) = ax + b
        x_ticks_linear_regression (np.ndarray): X values to calculate the linear regression
        real_dots (np.ndarray): Real dots to use to adapt the linear regression
        lines (tuple[Line2D, ...]): Lines with graphics
        slope (float, optional): The 'a' (weight) of the linear regression: f(x) = ax + b. Defaults to 0.0.
        tan_range (float, optional): Range of the tangent line. Defaults to 1.5.

    Returns:
        tuple[Line2D, ...]: All lines to update graphics
    """
    
    linear_regression, square_error_curve_tan, current_error_point, bias_text, error_text = lines

    y_points_linear_regression: np.ndarray = calcule_yPoints_linear_function(
        x=x_ticks_linear_regression,
        bias=current_bias,
        slope=slope
    )
    linear_regression.set_data(x_ticks_linear_regression, y_points_linear_regression)
    bias_text.set_text(f'Bias (intercept): {current_bias:.4f}')

    y_real: np.ndarray = real_dots[:, 1]
    x_real: np.ndarray = real_dots[:, 0]
    y_pred: np.ndarray = calcule_yPoints_linear_function(
        x=x_real, 
        bias=current_bias, 
        slope=slope
    )

    square_error: float = calcule_square_error(real_dots, current_bias, slope)
    x_tan, y_tan = calcule_tan_line(
        square_error=square_error,
        y_real=y_real,
        y_pred=y_pred,
        bias=current_bias,
        tan_range=tan_range
    )
    square_error_curve_tan.set_data(x_tan, y_tan)

    # actual curve point
    current_error_point.set_data([current_bias], [square_error])

    error_text.set_text(f'Square Error: {square_error:.4f}')

    return linear_regression, square_error_curve_tan, current_error_point, bias_text, error_text


## Constantes para controlar a regressão

In [ ]:
X_TICKS_LINEAR_REGRESSION: np.ndarray = np.linspace(0, 3.5)
X_TICKS_ERROR_CURVE: np.ndarray = np.linspace(0, 2)
REAL_DOTS: np.ndarray = np.array([[0.5, 1.4], [2.3, 1.9], [2.9, 3.2]])
INITIAL_BIAS: int = 0
SLOPE = 0.64
LEARN_RATE: float = 0.1
MAX_ITERATIONS: int = 150
TAN_RANGE: float = 1.5

## Pegando os valores iniciais

In [ ]:
y_points_linear_regression: np.ndarray = calcule_yPoints_linear_function(
    x=X_TICKS_LINEAR_REGRESSION,
    bias=INITIAL_BIAS,
    slope=SLOPE,
)

y_points_square_error: np.ndarray = calcule_error_curve_values(
    x_ticks=X_TICKS_ERROR_CURVE,
    slope=SLOPE,
    real_dots=REAL_DOTS,
)

## Criando a figura e os eixos

In [ ]:
fig, axes = create_subplots()
lines_to_update = config_axels(
    axes=axes,
    bias=INITIAL_BIAS,
    slope=SLOPE,
    real_dots=REAL_DOTS,
    x_ticks_linear_regression=X_TICKS_LINEAR_REGRESSION,
    ypoints_linear_regression=y_points_linear_regression,
    x_ticks_error_curve=X_TICKS_ERROR_CURVE,
    ypoints_square_error_curve=y_points_square_error,
)

bias_generator = gradient_descent_generator(
    initial_bias=INITIAL_BIAS,
    slope=SLOPE,
    real_dots=REAL_DOTS,
    learn_rate=LEARN_RATE,
    max_iterations=MAX_ITERATIONS
)

bias_frame_array: np.ndarray = np.array(list(bias_generator))

## Iniciando a animação

In [ ]:
from matplotlib.animation import FuncAnimation


ani = FuncAnimation(
    fig=fig,
    func=animate,
    frames=bias_frame_array,
    fargs=(
        X_TICKS_LINEAR_REGRESSION,
        REAL_DOTS,
        lines_to_update,
        SLOPE,
        TAN_RANGE,
    ),
    interval=100, # Interval between frames
    blit=True,    # Otmization: just draw the changes
    repeat=False  # Do not reapeat the animation
)


## Mostrando a animação

In [ ]:
from IPython.display import HTML

plt.close(fig)
HTML(ani.to_jshtml())